In [1]:
!pip install skorch

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
from skorch import NeuralNetClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
import torch.nn.functional as F
from torch import nn, optim

In [3]:
%run ../services/imports.py
%run ../services/visualization.py
from services.function import Function

In [4]:
df_poscomp = pd.read_csv("../dados/processed/dados_poscomp_completo_test.csv")

In [5]:
df_poscomp.head()

,idade,sexo,estado,presença,cargo,especialidade,matematica,fund_computacao,tec_computacao,ano,linhas_pesquisas,area_concentration
0,1989-11-29,masculino,ce,s,mestrado/doutorado,teoria da computação,14.0,21.0,10.0,2016,sistemas de informação,informatica
1,1995-02-23,masculino,rs,s,mestrado/doutorado,metodologias e técnicas de computação,16.0,22.0,12.0,2016,engenharia de software,informatica
2,1991-12-13,masculino,sp,s,mestrado/doutorado,engenharia de software,4.0,16.0,8.0,2016,engenharia de software,informatica
3,1986-09-29,masculino,pa,s,mestrado/doutorado,sistemas de informação,4.0,13.0,7.0,2016,sistemas de informação,informatica
4,1997-12-26,feminino,am,s,mestrado/doutorado,computação gráfica,6.0,15.0,11.0,2016,inteligência artificial,computacao


In [6]:
# converter os type das colunas
df_poscomp["ano"] = df_poscomp["ano"].astype("object")
# Primeiramente, será excluídos candidatos que realizaram o exame no Peru
df_poscomp = df_poscomp.query("estado != 'ou' & presença != 'n'")
df_poscomp["idade"] = df_poscomp.apply(Function.calculate_age, axis=1)
df_poscomp = df_poscomp.query('idade > 0')

In [7]:
df_poscomp

,idade,sexo,estado,presença,cargo,especialidade,matematica,fund_computacao,tec_computacao,ano,linhas_pesquisas,area_concentration
0,27,masculino,ce,s,mestrado/doutorado,teoria da computação,14.0,21.0,10.0,2016,sistemas de informação,informatica
1,21,masculino,rs,s,mestrado/doutorado,metodologias e técnicas de computação,16.0,22.0,12.0,2016,engenharia de software,informatica
2,25,masculino,sp,s,mestrado/doutorado,engenharia de software,4.0,16.0,8.0,2016,engenharia de software,informatica
3,30,masculino,pa,s,mestrado/doutorado,sistemas de informação,4.0,13.0,7.0,2016,sistemas de informação,informatica
4,19,feminino,am,s,mestrado/doutorado,computação gráfica,6.0,15.0,11.0,2016,inteligência artificial,computacao
...,...,...,...,...,...,...,...,...,...,...,...,...
13470,22,feminino,mg,s,mestrado/doutorado,inteligência artificial,14.0,16.0,8.0,2019,inteligência artificial,computacao
13471,48,masculino,pb,s,mestrado/doutorado,redes de computadores,10.0,16.0,9.0,2019,redes de computadores,informatica
13472,24,masculino,sp,s,mestrado/doutorado,engenharia de software,9.0,18.0,8.0,2019,engenharia de software,informatica
13473,25,masculino,es,s,mestrado/doutorado,inteligência artificial,10.0,16.0,6.0,2019,inteligência artificial,computacao


In [35]:
X_dataset = df_poscomp.drop(columns=['ano', 'especialidade'])

In [36]:
X_dataset['linhas_pesquisas'].value_counts()

linhas_pesquisas
inteligência artificial    3663
sistemas de computação     2472
engenharia de software     2463
redes de computadores      1228
sistemas de informação     1150
Name: count, dtype: int64

In [10]:
def encode_categorical_columns(df, columns):
    le = LabelEncoder()
    for col in columns:
        df[col] = le.fit_transform(df[col])
    return df

categorical_columns = ['linhas_pesquisas', 'sexo', ]
X_dataset = encode_categorical_columns(X_dataset, categorical_columns)

In [11]:
X = X_dataset.drop(['linhas_pesquisas'], axis=1).values
y = X_dataset['linhas_pesquisas'].values

In [12]:
X

array([[14., 21., 10.],
       [16., 22., 12.],
       [ 4., 16.,  8.],
       ...,
       [ 9., 18.,  8.],
       [10., 16.,  6.],
       [13., 17.,  5.]])

In [13]:
X.shape

(10976, 3)

In [14]:
y.shape

(10976,)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
np.unique(y_train)

array([0, 1, 2, 3, 4])

In [17]:
X_train = torch.tensor(X_train, dtype = torch.float)
y_train = torch.tensor(y_train, dtype = torch.long)

In [29]:
class classificador_torch(nn.Module):
    def __init__(self, activation, neurons, initializer):
        super().__init__()
        
        self.dense0 = nn.Linear(3, neurons)
        initializer(self.dense0.weight)
        self.activation0 = activation
        
        self.dense1 = nn.Linear(neurons, neurons)
        initializer(self.dense1.weight)
        self.activation1 = activation

        self.dense2 = nn.Linear(neurons, neurons)  # Nova camada oculta
        initializer(self.dense2.weight)
        self.activation2 = activation

        self.dense3 = nn.Linear(neurons, neurons)  # Outra nova camada
        initializer(self.dense3.weight)
        self.activation3 = activation
        
        self.dense4 = nn.Linear(neurons, 5)  # Camada de saída (3 classes)
        
        # self.dropout = nn.Dropout(dropout)
        
        self.output = nn.Softmax()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        # X = self.dropout(X)

        X = self.dense1(X)
        X = self.activation1(X)
        # X = self.dropout(X)

        X = self.dense2(X)
        X = self.activation2(X)
        # X = self.dropout(X)

        X = self.dense3(X)
        X = self.activation3(X)
        # X = self.dropout(X)

        X = self.dense4(X)  # Saída final sem ativação (CrossEntropyLoss espera logits)
        X = self.output(X)
        return X

In [30]:
# Hiperparâmetros do modelo
activation = F.relu
initializer = nn.init.uniform_
neurons = 50
# dropout_rate = 0.2

# Criando o classificador skorch
classificador_sklearn = NeuralNetClassifier(
    module=classificador_torch,  # Passa a classe, não uma instância
    module__activation=activation,
    module__neurons=neurons,
    module__initializer=initializer,
    # module__dropout=dropout_rate,
    criterion=torch.nn.CrossEntropyLoss,
    train_split=False,
    optimizer=optim.Adam,
    optimizer__weight_decay=0.0001,
    max_epochs=100,  # Número de épocas
    batch_size=32
)

print(classificador_sklearn)

<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.classificador_torch'>,
  module__activation=<function relu at 0x78158dbcad40>,
  module__initializer=<function uniform_ at 0x781580b69800>,
  module__neurons=50,
)


In [31]:
resultados = cross_val_score(classificador_sklearn, X_train, y_train, cv = 5,
                             scoring='accuracy')

  epoch    train_loss     dur
-------  ------------  ------
      1        1.6767  0.2783
      2        1.5521  0.2027
      3        1.5486  0.2028
      4        1.5474  0.2024
      5        1.5470  0.2038
      6        1.5468  0.2053
      7        1.5469  0.2153
      8        1.5467  0.2339
      9        1.5468  0.2623
     10        1.5466  0.2617
     11        1.5468  0.2640
     12        1.5468  0.2636
     13        1.5468  0.2532
     14        1.5466  0.2606
     15        1.5469  0.2632
     16        1.5467  0.2622
     17        1.5468  0.2523
     18        1.5468  0.2542
     19        1.5468  0.2587
     20        1.5468  0.2592
     21        1.5468  0.2649
     22        1.5467  0.2629
     23        1.5468  0.2584
     24        1.5467  0.2641
     25        1.5468  0.2739
     26        1.5467  0.2729
     27        1.5468  0.2847
     28        1.5470  0.3005
     29        1.5467  0.2923
     30        1.5467  0.2836
     31        1.5467  0.2753
     32   

In [32]:
resultados

array([0.33116461, 0.33181522, 0.33116461, 0.33138021, 0.33138021])